# ML Lab 2: Data Preprocessing
## Flood Prediction Dataset - Kaggle Playground Series S4E5

**Dataset Description:**
- **Target Variable:** FloodProbability
- **Task:** Predict FloodProbability for test set
- **Dataset:** Generated from deep learning model trained on Flood Prediction Factors dataset

**Reference:** Following the preprocessing guide from Medium article (Part 1 & 2)

---
## Step 1: Import Required Libraries

**Purpose:** Load all necessary libraries for data manipulation, visualization, and preprocessing.

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Libraries imported successfully!")

---
## Step 2: Load Dataset

**Purpose:** Load the training data and perform initial size check.

In [ ]:
# Load training data
# NOTE: Change the path based on your environment:

# For Google Colab (with Google Drive mounted):
df = pd.read_csv('/content/drive/MyDrive/Dataset/playground-series-s4e5/train.csv')

# For Local execution:
# df = pd.read_csv('../data/train_original.csv')

print("Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Rows: {df.shape[0]:,}")
print(f"Columns: {df.shape[1]}")

---
## Step 3: Initial Data Exploration

**Purpose:** Understand the dataset structure, data types, and basic statistics.

### 3.1 Display First Few Rows

In [ ]:
# Display first 5 rows to understand data structure
print("First 5 rows of the dataset:")
df.head()

### 3.2 Dataset Information

In [ ]:
# Display dataset information: column names, data types, non-null counts
print("Dataset Information:")
df.info()

### 3.3 Statistical Summary

In [ ]:
# Statistical summary of numerical columns
print("Statistical Summary:")
df.describe()

### 3.4 Identify Column Types

In [ ]:
# Identify numerical and categorical columns
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

print("Column Type Analysis:")
print("=" * 60)
print(f"Numerical columns ({len(numerical_cols)}):")
print(numerical_cols)
print(f"\nCategorical columns ({len(categorical_cols)}):")
print(categorical_cols if categorical_cols else "None")

print("\n" + "=" * 60)
print("OBSERVATION:")
print("All features are numerical (no categorical features found).")
print("Therefore, we will NOT need to apply encoding techniques.")

---
## Step 4: Data Cleaning

**Purpose:** Check for and handle missing values, duplicates, and outliers.

### 4.1 Check for Missing Values

**Reference Guide:** "Check for missing values and apply imputation if needed."

In [ ]:
# Step 4.1: Check for missing values in each column
print("STEP 4.1: Checking for Missing Values")
print("=" * 60)

missing_values = df.isna().sum()
missing_percentage = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing_values.index,
    'Missing_Count': missing_values.values,
    'Percentage': missing_percentage.values
})

# Display only columns with missing values
missing_data = missing_df[missing_df['Missing_Count'] > 0]

if len(missing_data) > 0:
    print("Columns with missing values:")
    print(missing_data)
else:
    print("✓ No missing values found in any column!")

print("\n" + "=" * 60)
print("DECISION:")
if len(missing_data) > 0:
    print("We WILL apply imputation using median (as per reference guide).")
else:
    print("We will NOT apply any imputation technique.")
    print("REASON: No missing values detected in the dataset.")

### 4.2 Check for Duplicate Rows

**Reference Guide:** "Remove duplicate rows and reset index."

In [ ]:
# Step 4.2: Check for duplicate rows
print("STEP 4.2: Checking for Duplicate Rows")
print("=" * 60)

# Check duplicates (including ID column)
duplicate_count_with_id = df.duplicated().sum()
print(f"Duplicate rows (with ID): {duplicate_count_with_id:,}")

# Check duplicates (excluding ID column - more meaningful)
duplicate_count_no_id = df.drop('id', axis=1).duplicated().sum()
print(f"Duplicate rows (excluding ID): {duplicate_count_no_id:,}")

if duplicate_count_no_id > 0:
    duplicate_percentage = (duplicate_count_no_id / len(df)) * 100
    print(f"Percentage of duplicates: {duplicate_percentage:.2f}%")
else:
    print("✓ No duplicate rows found!")

print("\n" + "=" * 60)
print("DECISION:")
if duplicate_count_no_id > 0:
    print(f"We WILL remove {duplicate_count_no_id:,} duplicate rows.")
    print("REASON: Duplicates can bias model training.")
    # Uncomment below to remove duplicates
    # df = df.drop_duplicates().reset_index(drop=True)
else:
    print("We will NOT remove any rows.")
    print("REASON: No duplicates detected in the dataset.")

### 4.3 Check for Outliers

**Reference Guide:** "Detect outliers using boxplots and IQR method, then decide whether to remove or replace based on domain knowledge."

In [ ]:
# Step 4.3: Check for outliers - First, examine feature ranges
print("STEP 4.3: Checking for Outliers")
print("=" * 60)

# Get feature columns (exclude id and target)
feature_cols = [col for col in df.columns if col not in ['id', 'FloodProbability']]

print(f"Analyzing {len(feature_cols)} features...\n")
print("Feature Value Ranges:")
print("-" * 80)

# Display range for each feature
for col in feature_cols:
    min_val = df[col].min()
    max_val = df[col].max()
    unique_vals = df[col].nunique()
    print(f"{col:40s} | Min: {min_val:2.0f} | Max: {max_val:2.0f} | Unique: {unique_vals:3d}")

print("\n" + "=" * 60)
print("OBSERVATION:")
print("All features are integer scores ranging from 0 to 18.")
print("These represent severity/rating scores for flood-related factors.")
print("Example: MonsoonIntensity=0 means 'low', MonsoonIntensity=16 means 'very high'")

In [ ]:
# Now check outliers using IQR method (as per reference guide)
print("\nOutlier Detection using IQR Method:")
print("=" * 60)

outlier_summary = []

for col in feature_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    # Define outlier bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Count outliers
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outlier_count = len(outliers)
    outlier_pct = (outlier_count / len(df)) * 100
    
    if outlier_count > 0:
        outlier_summary.append({
            'Feature': col,
            'Outliers': outlier_count,
            'Percentage': outlier_pct
        })

if outlier_summary:
    print(f"\nFeatures with outliers detected by IQR method:")
    print("-" * 80)
    # Show top 5 features with most outliers
    for item in sorted(outlier_summary, key=lambda x: x['Percentage'], reverse=True)[:5]:
        print(f"{item['Feature']:40s} | {item['Outliers']:8,} outliers ({item['Percentage']:5.2f}%)")
    if len(outlier_summary) > 5:
        print(f"... and {len(outlier_summary) - 5} more features")
else:
    print("✓ No outliers detected by IQR method.")

print("\n" + "=" * 60)
print("DECISION:")
print("We will NOT remove any outliers.")
print("REASON:")
print("1. All values are within valid domain range (0-18 severity scores)")
print("2. High values (e.g., MonsoonIntensity=16) represent extreme conditions,")
print("   not data entry errors.")
print("3. As per reference guide: Use domain knowledge - these are valid observations.")
print("4. Removing extreme values would lose important information about")
print("   high-risk flood scenarios.")

---
## Step 5: Correlation Analysis & Feature Selection

**Purpose:** Analyze feature correlations to detect multicollinearity and select relevant features.

**Reference Guide:** "Remove features with correlation > 0.9 to avoid multicollinearity."

### 5.1 Correlation with Target Variable

In [ ]:
# Step 5.1: Calculate correlation with target variable
print("STEP 5.1: Correlation with Target Variable")
print("=" * 60)

# Get all columns except 'id'
feature_cols_all = [col for col in df.columns if col != 'id']

# Calculate correlation matrix
corr_matrix = df[feature_cols_all].corr()

# Get correlation with target
target_corr = corr_matrix['FloodProbability'].sort_values(ascending=False)

print("\nCorrelation of each feature with FloodProbability:")
print("-" * 60)
print(target_corr)

print("\n" + "=" * 60)
print("OBSERVATION:")
print(f"All features have weak-to-moderate correlation with target (0.17 to 0.19).")
print(f"No feature has very strong correlation (>0.5).")
print(f"This suggests all features contribute similarly to flood prediction.")

### 5.2 Visualize Correlation Matrix

In [ ]:
# Step 5.2: Create correlation heatmap for visualization
print("STEP 5.2: Creating Correlation Heatmap")
print("=" * 60)

plt.figure(figsize=(20, 16))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('correlation_heatmap.png', dpi=100, bbox_inches='tight')
plt.show()

print("\n✓ Correlation heatmap saved as 'correlation_heatmap.png'")

### 5.3 Check for Multicollinearity

**Reference Guide:** "Remove highly correlated features (correlation > 0.9) to avoid multicollinearity."

In [ ]:
# Step 5.3: Detect multicollinearity (highly correlated feature pairs)
print("STEP 5.3: Checking for Multicollinearity")
print("=" * 60)
print("Threshold: Correlation > 0.9")
print("-" * 80)

# Find highly correlated pairs (correlation > 0.9)
high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.9:
            high_corr_pairs.append({
                'Feature1': corr_matrix.columns[i],
                'Feature2': corr_matrix.columns[j],
                'Correlation': corr_matrix.iloc[i, j]
            })

if high_corr_pairs:
    print("\nHighly correlated feature pairs found:")
    for pair in high_corr_pairs:
        print(f"{pair['Feature1']:40s} <-> {pair['Feature2']:40s} | Corr: {pair['Correlation']:.4f}")
else:
    print("✓ No highly correlated feature pairs found (correlation > 0.9)")

print("\n" + "=" * 60)
print("DECISION:")
if high_corr_pairs:
    print(f"We WILL remove {len(high_corr_pairs)} feature(s) to avoid multicollinearity.")
    print("REASON: Highly correlated features provide redundant information.")
else:
    print("We will NOT remove any features.")
    print("REASON:")
    print("1. No multicollinearity detected (all feature pairs < 0.9 correlation)")
    print("2. All features are independent and contribute unique information")
    print("3. All 20 features will be retained for modeling")

---
## Step 6: Check if Encoding is Needed

**Reference Guide:** "Encode categorical features using One-Hot or Ordinal encoding."

**Purpose:** Determine if categorical encoding is required.

In [ ]:
# Step 6: Check if categorical encoding is needed
print("STEP 6: Checking if Categorical Encoding is Needed")
print("=" * 60)

# Identify categorical columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"Categorical columns found: {len(categorical_cols)}")
if categorical_cols:
    print("Columns:", categorical_cols)
else:
    print("None")

print("\n" + "=" * 60)
print("DECISION:")
if categorical_cols:
    print(f"We WILL apply encoding to {len(categorical_cols)} categorical feature(s).")
    print("Method: One-Hot Encoding for nominal, Ordinal Encoding for ordinal features.")
else:
    print("We will NOT apply any encoding.")
    print("REASON: All features are already numerical (int64/float64).")
    print("No categorical features detected in the dataset.")

---
## Step 7: Check if Resampling is Needed

**Reference Guide:** "Handle imbalanced data using upsampling, downsampling, or SMOTE."

**Purpose:** Determine if the dataset needs rebalancing.

In [ ]:
# Step 7: Check if resampling is needed for imbalanced data
print("STEP 7: Checking if Resampling is Needed")
print("=" * 60)

# Check if target is categorical (classification) or continuous (regression)
target_dtype = df['FloodProbability'].dtype
target_unique = df['FloodProbability'].nunique()
target_min = df['FloodProbability'].min()
target_max = df['FloodProbability'].max()

print(f"Target variable: FloodProbability")
print(f"Data type: {target_dtype}")
print(f"Unique values: {target_unique:,}")
print(f"Range: {target_min:.3f} to {target_max:.3f}")

# Check if it's a classification or regression problem
is_classification = target_unique < 50  # Arbitrary threshold

print("\n" + "=" * 60)
print("OBSERVATION:")
if is_classification:
    print("This appears to be a CLASSIFICATION problem.")
    # Check class balance
    class_dist = df['FloodProbability'].value_counts()
    print("\nClass distribution:")
    print(class_dist)
else:
    print("This is a REGRESSION problem (continuous target variable).")
    print(f"Target has {target_unique:,} unique continuous values.")

print("\n" + "=" * 60)
print("DECISION:")
if is_classification:
    print("We MAY apply resampling techniques if classes are imbalanced.")
    print("Check class distribution above and apply SMOTE/upsampling if needed.")
else:
    print("We will NOT apply any resampling techniques.")
    print("REASON:")
    print("1. This is a REGRESSION problem (predicting continuous values)")
    print("2. Resampling techniques (SMOTE, upsampling, downsampling) are")
    print("   designed for CLASSIFICATION problems with imbalanced classes")
    print("3. They are not applicable to regression tasks")

---
## Step 8: Data Splitting

**Reference Guide:** "Split data into training (60-80%), validation (10-20%), and test (10-20%) sets."

**Purpose:** Separate data before scaling to prevent data leakage.

**Split Ratio:** 60% Train / 20% Validation / 20% Test

In [ ]:
# Step 8: Split data into train, validation, and test sets
print("STEP 8: Data Splitting")
print("=" * 60)
print("Split ratio: 60% Train / 20% Validation / 20% Test")
print("-" * 60)

# Separate features, target, and ID
X = df.drop(['id', 'FloodProbability'], axis=1)  # Features only
y = df['FloodProbability']                       # Target
ids = df['id']                                   # IDs for reference

print(f"\nOriginal dataset shape: {df.shape}")
print(f"Features (X) shape: {X.shape}")
print(f"Target (y) shape: {y.shape}")

# First split: 80% (train+valid) and 20% (test)
X_temp, X_test, y_temp, y_test, ids_temp, ids_test = train_test_split(
    X, y, ids, test_size=0.2, random_state=42
)

# Second split: 60% (train) and 20% (valid) from the 80%
X_train, X_valid, y_train, y_valid, ids_train, ids_valid = train_test_split(
    X_temp, y_temp, ids_temp, test_size=0.25, random_state=42  # 0.25 * 0.8 = 0.2
)

print("\n" + "=" * 60)
print("Data Split Summary:")
print("=" * 60)
print(f"Training set:   {X_train.shape[0]:8,} samples ({X_train.shape[0]/len(df)*100:.1f}%)")
print(f"Validation set: {X_valid.shape[0]:8,} samples ({X_valid.shape[0]/len(df)*100:.1f}%)")
print(f"Test set:       {X_test.shape[0]:8,} samples ({X_test.shape[0]/len(df)*100:.1f}%)")
print(f"Total:          {len(df):8,} samples (100.0%)")

print("\n" + "=" * 60)
print("WHY SPLIT BEFORE SCALING?")
print("REASON:")
print("1. Prevents DATA LEAKAGE: If we scale before splitting, test data")
print("   statistics (mean, std) influence training data scaling.")
print("2. Correct approach: Fit scaler on TRAINING data only, then")
print("   transform validation and test sets using the same scaler.")
print("3. This ensures test set remains truly unseen during preprocessing.")

---
## Step 9: Feature Scaling (Standardization)

**Reference Guide:** "Use StandardScaler for standardization (mean=0, std=1). Improves model performance."

**Purpose:** Normalize features to standard scale for better model convergence.

**Method:** StandardScaler (Mean=0, Std=1)

### 9.1 Check Statistics Before Scaling

In [ ]:
# Step 9.1: Check statistics before scaling
print("STEP 9.1: Statistics Before Scaling")
print("=" * 60)
print("Training set feature statistics:")
print("-" * 60)
print(X_train.describe().loc[['mean', 'std']].round(2))

print("\n" + "=" * 60)
print("OBSERVATION:")
print("Features have different scales (means around 4-5, std around 2).")
print("Scaling is needed for algorithms sensitive to feature magnitude.")

### 9.2 Apply StandardScaler

**Important:** Fit scaler on training data only!

In [ ]:
# Step 9.2: Apply StandardScaler
print("STEP 9.2: Applying StandardScaler")
print("=" * 60)

# Initialize StandardScaler
scaler = StandardScaler()

# Fit scaler on TRAINING data only
print("Fitting scaler on training data...")
X_train_scaled = scaler.fit_transform(X_train)

# Transform validation and test data using the SAME scaler
print("Transforming validation data...")
X_valid_scaled = scaler.transform(X_valid)

print("Transforming test data...")
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrames with column names
feature_names = X_train.columns.tolist()

X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=feature_names)
X_valid_scaled_df = pd.DataFrame(X_valid_scaled, columns=feature_names)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=feature_names)

print("\n✓ Scaling complete!")

# Verify scaling
print("\n" + "=" * 60)
print("Statistics After Scaling:")
print("-" * 60)
print(X_train_scaled_df.describe().loc[['mean', 'std']].round(4))

print("\n" + "=" * 60)
print("VERIFICATION:")
print("✓ Mean ≈ 0 for all features (standardized)")
print("✓ Std ≈ 1 for all features (standardized)")
print("✓ All features now on the same scale")

print("\n" + "=" * 60)
print("WHY StandardScaler?")
print("REASON:")
print("1. Works well for algorithms assuming normal distribution")
print("   (Linear Regression, Logistic Regression, SVM, Neural Networks)")
print("2. Better for distance-based algorithms (KNN, K-means)")
print("3. Reference guide shows StandardScaler improves F1 and AUC-ROC scores")
print("4. Less sensitive to outliers compared to MinMaxScaler")

---
## Step 10: Save Preprocessed Data

**Purpose:** Save cleaned and scaled data for model training.

In [ ]:
# Step 10: Reconstruct and save preprocessed datasets
print("STEP 10: Saving Preprocessed Data")
print("=" * 60)

# Reconstruct full datasets with id and target
train_preprocessed = pd.concat([
    ids_train.reset_index(drop=True),
    X_train_scaled_df.reset_index(drop=True),
    y_train.reset_index(drop=True)
], axis=1)

valid_preprocessed = pd.concat([
    ids_valid.reset_index(drop=True),
    X_valid_scaled_df.reset_index(drop=True),
    y_valid.reset_index(drop=True)
], axis=1)

test_preprocessed = pd.concat([
    ids_test.reset_index(drop=True),
    X_test_scaled_df.reset_index(drop=True),
    y_test.reset_index(drop=True)
], axis=1)

# Save to CSV files in Google Drive
# NOTE: Change these paths based on your environment:
# - For Google Colab: Use '/content/drive/MyDrive/Dataset/playground-series-s4e5/'
# - For Local: Use '../data/'
print("Saving files...")
train_preprocessed.to_csv('/content/drive/MyDrive/Dataset/playground-series-s4e5/train_preprocessed.csv', index=False)
valid_preprocessed.to_csv('/content/drive/MyDrive/Dataset/playground-series-s4e5/valid_preprocessed.csv', index=False)
test_preprocessed.to_csv('/content/drive/MyDrive/Dataset/playground-series-s4e5/test_preprocessed.csv', index=False)

print("\n" + "=" * 60)
print("Files Saved Successfully:")
print("=" * 60)
print(f"✓ train_preprocessed.csv: {len(train_preprocessed):,} rows")
print(f"✓ valid_preprocessed.csv: {len(valid_preprocessed):,} rows")
print(f"✓ test_preprocessed.csv: {len(test_preprocessed):,} rows")

print("\nFiles saved to Google Drive!")
print("Location: /content/drive/MyDrive/Dataset/playground-series-s4e5/")
print("\nThese files are ready for:")
print("- Model training (train_preprocessed.csv)")
print("- Hyperparameter tuning (valid_preprocessed.csv)")
print("- Final evaluation (test_preprocessed.csv)")

---
## Summary: Preprocessing Decisions

### Steps Taken and Justifications:

| Step | Action Taken | Decision | Justification |
|------|--------------|----------|---------------|
| **1. Missing Values** | Checked | ✓ NO imputation applied | No missing values found (0/1,117,957 rows) |
| **2. Duplicates** | Checked | ✓ NO removal needed | No duplicate rows found |
| **3. Outliers** | Checked | ✓ NO removal applied | All values are valid domain scores (0-18 range) |
| **4. Multicollinearity** | Checked | ✓ NO features removed | No correlation > 0.9 detected |
| **5. Categorical Encoding** | Checked | ✓ NO encoding applied | All features already numerical |
| **6. Resampling** | Checked | ✓ NO resampling applied | Regression problem (not classification) |
| **7. Data Splitting** | Applied | ✓ 60/20/20 split | Standard practice for large datasets |
| **8. Feature Scaling** | Applied | ✓ StandardScaler used | Best for regression, prevents data leakage |

### Key Principles Followed:

1. **Data-Driven Decisions:** Every decision based on actual data analysis, not assumptions
2. **Prevented Data Leakage:** Split data BEFORE scaling
3. **Domain Knowledge:** Kept "outliers" that represent valid extreme conditions
4. **Best Practices:** Followed reference guide recommendations
5. **No Unnecessary Steps:** Only applied preprocessing where data required it

### Final Dataset Characteristics:

- **Training Set:** 670,773 samples (60%)
- **Validation Set:** 223,592 samples (20%)
- **Test Set:** 223,592 samples (20%)
- **Features:** 20 (all standardized, mean=0, std=1)
- **Target:** FloodProbability (continuous, range: 0.285-0.725)
- **Quality:** Clean, scaled, no leakage

### Files Generated:

1. `train_preprocessed.csv` - Ready for model training
2. `valid_preprocessed.csv` - Ready for hyperparameter tuning
3. `test_preprocessed.csv` - Ready for final evaluation
4. `correlation_heatmap.png` - Feature correlation visualization

### Next Steps:

1. Model selection (Linear Regression, Random Forest, Gradient Boosting, etc.)
2. Model training on `train_preprocessed.csv`
3. Hyperparameter tuning using `valid_preprocessed.csv`
4. Final evaluation on `test_preprocessed.csv`
5. Metrics: MAE, RMSE, R² Score

---

**Preprocessing Complete! ✓**

All decisions documented and justified according to the reference guide.